# Prediction Difference Computation Between Age Group

Logic: 10 Folder Prediction on self-model vs average other groups

In [1]:
%pwd

'/mnt/d/OneDrive - Kyushu University/ESG09_Article/Code'

In [2]:
%cd ..

/mnt/d/OneDrive - Kyushu University/ESG09_Article


/home/linux/anaconda3/envs/automl/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Import Package

In [79]:
from joblib import dump, load
import numpy as np
import os 
import pandas as pd
import random
from scipy import stats
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb

## Load dataset and divide them into subgroup

In [65]:
Df_Filename = os.path.join("Data", "GallupWB_Ml64var1911k14wave_v1.parquet")

In [66]:
Df = pd.read_parquet(Df_Filename)

In [67]:
Df.shape

(1911212, 64)

In [68]:
Df.columns

Index(['wave', 'INCOME_2', 'Cantril_ladder', 'Health_disable', 'Relative_have',
       'Living_standard_change', 'Enough_food', 'Enough_shelter',
       'Well_rested', 'Respected', 'Smile', 'Interesting_thing', 'Enjoyment',
       'Physical_pain', 'Worry', 'Sadness', 'Stress', 'Anger',
       'City_satisficied', 'Economic_change', 'Goodtime_job', 'Sat_pubtran',
       'Sat_road', 'Sat_edu', 'Sat_qualityair', 'Sat_qualitywater',
       'Sat_healthcare', 'Sat_affhouse', 'Sat_oppofriend', 'Good_minorities',
       'Good_gayles', 'Good_immigrants', 'Donated', 'Volunteer',
       'Help_stranger', 'Voice_official', 'Local_police', 'Safety_walk',
       'Stolen', 'Assualted', 'Religion_importance', 'Children_respected',
       'Children_learn', 'Women_respected', 'Sat_dealpoor', 'Sat_perserveenv',
       'Freedom_chooselife', 'Conf_military', 'Conf_judicial',
       'Conf_government', 'Conf_financial', 'Conf_honestyelections',
       'Freedom_media', 'Corruption_business', 'Corruption_governm

In [70]:
Df.sort_index().head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
367305,4,1.351047e+01,6.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,21.0,1.0,1.0,0.0,2.0,1.0,USA
367306,4,2.296780e+06,6.0,2.0,1.0,-0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,34.0,1.0,1.0,0.0,2.0,5.0,USA
367307,4,3.512722e+05,8.0,2.0,1.0,-1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,44.0,1.0,1.0,0.0,2.0,5.0,USA
367308,4,5.404188e+04,8.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,1.0,1.0,67.0,1.0,1.0,0.0,1.0,5.0,USA
367309,4,0.000000e+00,8.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,49.0,2.0,1.0,0.0,1.0,1.0,USA


In [38]:
Df = Df.sample(frac=1, random_state=42)

In [45]:
Df.head()

,wave,INCOME_2,Cantril_ladder,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,...,Corruption_government,Performance_leadership,Gender_female,Age,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1469152,10,35602.470662,7.0,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,64.0,1.0,5.0,0.0,2.0,3.0,NZL
1562723,11,20170.734058,5.0,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,18.0,1.0,6.0,0.0,2.0,1.0,LBN
708695,6,8664.458086,4.0,2.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,15.0,1.0,6.0,1.0,3.0,3.0,BWA
1828455,12,30706.925977,2.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,33.0,2.0,1.0,1.0,2.0,1.0,KWT
968859,7,4826.805232,8.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.0,29.0,8.0,1.0,1.0,3.0,4.0,HND


In [39]:
Df['COUNTRY_ISO3'] = Df['COUNTRY_ISO3'].astype('category')

In [40]:
Df.isna().any().any()

False

In [41]:
Df_young = Df.loc[Df['Age']<=40, :].drop(columns=['Age'])

In [42]:
Df_middle = Df.loc[(Df['Age']>40)&(Df['Age']<=65), :].drop(columns=['Age'])

In [43]:
Df_old = Df.loc[Df['Age']>65, :].drop(columns=['Age'])

In [99]:
yy = Df_young['Cantril_ladder']
Xy = Df_young.drop(columns=['Cantril_ladder'])

In [100]:
ym = Df_middle['Cantril_ladder']
Xm = Df_middle.drop(columns=['Cantril_ladder'])

In [101]:
yo = Df_old['Cantril_ladder']
Xo = Df_old.drop(columns=['Cantril_ladder'])

## Prediction of Each Group Based on Different Models

### Young Hyperparameter Reload, build model, and conduct 10 fold 10 times CV

In [16]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [17]:
bayes_search_young = load('Results/BayesSearchAgeYoung20iter.joblib')

In [18]:
print(f"Best parameters: {bayes_search_young.best_params_}")
print(f"Best score: {bayes_search_young.best_score_}")

Best parameters: OrderedDict([('gamma', 6.6360850776123375), ('learning_rate', 0.02519085390684862), ('max_delta_step', 3.0662510776734475), ('max_depth', 8), ('min_child_weight', 0.03343944487838535), ('n_estimators', 1136), ('reg_alpha', 0.001000562456331363), ('reg_lambda', 0.030390787275205566), ('subsample', 0.7971894567956928)])
Best score: 0.3522986332513942


In [20]:
dict(bayes_search_young.best_params_)

{'gamma': 6.6360850776123375,
 'learning_rate': 0.02519085390684862,
 'max_delta_step': 3.0662510776734475,
 'max_depth': 8,
 'min_child_weight': 0.03343944487838535,
 'n_estimators': 1136,
 'reg_alpha': 0.001000562456331363,
 'reg_lambda': 0.030390787275205566,
 'subsample': 0.7971894567956928}

In [30]:
young_xgb = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True, kwargs = dict(bayes_search_young.best_params_))

In [80]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [31]:
scores = cross_val_score(young_xgb, Xy, yy, cv=kf, scoring='r2')

In [32]:
scores

array([0.34465685, 0.33743302, 0.34463848, 0.3381389 , 0.34324076,
       0.33889873, 0.34312722, 0.3420818 , 0.34255657, 0.34565653])

In [47]:
Xy.head()

,wave,INCOME_2,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,Smile,...,Corruption_business,Corruption_government,Performance_leadership,Gender_female,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1562723,11,20170.734058,2.0,1.0,-1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,1.0,6.0,0.0,2.0,1.0,LBN
708695,6,8664.458086,2.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,1.0,6.0,1.0,3.0,3.0,BWA
1828455,12,30706.925977,2.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,1.0,KWT
968859,7,4826.805232,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,1.0,0.0,0.0,8.0,1.0,1.0,3.0,4.0,HND
2375401,16,6999.233312,2.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,6.0,0.0,2.0,4.0,THA


In [63]:
yy.head()

1562723    5.0
708695     4.0
1828455    2.0
968859     8.0
2375401    5.0
Name: Cantril_ladder, dtype: float64

In [147]:
random.seed(42)

In [148]:
range_limit = 8192
num_samples = 10
sampled_integers = random.sample(range(range_limit), num_samples)
print(sampled_integers)

[1824, 409, 4506, 4012, 3657, 2286, 1679, 1424, 6912, 520]


In [149]:
X = Xy
y = yy
model = young_xgb

In [150]:
m_prediction_list = [np.array(ym.index), np.array(ym.to_list())]
o_prediction_list = [np.array(yo.index), np.array(yo.to_list())]
fold_predictions_array_list = []
r2_scores_list = []
r2m_scores_list = []
r2o_scores_list = []

for i, random_integer in enumerate(sampled_integers):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_integer)
    
    r2_scores = []
    r2m_scores = []
    r2o_scores = []
    fold_predictions = []
    # Perform cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        this_folder = np.array([y_test.index, y_test, y_pred]).T
        fold_predictions.append(this_folder)
    
        # Middle Predict
        ym_pred = model.predict(Xm)
        m_prediction_list.append(ym_pred)
        r2m = r2_score(ym, ym_pred)
        r2m_scores.append(r2m)
        
        # Old Predict
        yo_pred = model.predict(Xo)
        o_prediction_list.append(yo_pred)
        r2o = r2_score(yo, yo_pred)
        r2o_scores.append(r2o)
    
    print(f"Mean R2: {np.mean(r2_scores)}")
    print(f"Standard Deviation of R2: {np.std(r2_scores)}")
    print(f"R2 list: {r2_scores}")
    print(f"R2 Midlle list: {r2m_scores}")
    print(f"R2 Old list: {r2o_scores}")
    
    fold_predictions_array = pd.DataFrame(np.concatenate(fold_predictions, axis = 0), columns = ['index', 'Real_y', f'Predict_y_{i}'])
    fold_predictions_array_list.append(fold_predictions_array)
    r2_scores_list.append(r2_scores)
    r2m_scores_list.append(r2m_scores)
    r2o_scores_list.append(r2o_scores)

Mean R2: 0.3418681327770883
Standard Deviation of R2: 0.0021637306331292985
R2 list: [0.337661600738246, 0.344733168535894, 0.3401356580778677, 0.34080624412681026, 0.3413704316102928, 0.34333532779188203, 0.3436211193595924, 0.34394325186598185, 0.3396951870940246, 0.34337933857029135]
R2 Midlle list: [0.39003536631725044, 0.3900359676795627, 0.389268573227073, 0.3899848612384408, 0.3902267191761808, 0.3903151185084013, 0.3899861158106628, 0.38947997713877625, 0.3899450096487722, 0.38952919377969264]
R2 Old list: [0.38064412555933436, 0.38170495698516116, 0.38031002714562523, 0.38075630445553277, 0.3813606248941598, 0.38143873830647, 0.3824145052589, 0.3809861373356538, 0.38144285553329715, 0.38152060473686666]
Mean R2: 0.34177202424582215
Standard Deviation of R2: 0.002973745774281953
R2 list: [0.338851541873059, 0.3370646801811291, 0.34672674805851345, 0.3412705024488537, 0.3452448128900061, 0.3376547094346015, 0.34309268743098364, 0.342822423632571, 0.342626471283767, 0.34236566522

### Young Model Middle Prediction

Prediction is average of 10 fold * 10 times

In [151]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
middle_preidiction_df = pd.DataFrame(np.array(m_prediction_list).T, columns = column_list)

In [152]:
middle_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1469152.0,7.0,6.447165,6.536165,6.521744,6.288181,6.397206,6.300893,6.548620,6.506389,...,6.363725,6.426962,6.342377,6.511581,6.189389,6.336519,6.486539,6.593062,6.485834,6.593565
1,831522.0,5.0,3.368080,3.341169,3.464390,3.124157,3.292077,3.159094,3.333278,3.349346,...,3.278545,3.468917,3.549992,3.247620,3.156353,3.699266,3.108299,3.158670,3.140763,3.729666
2,1938251.0,2.0,4.455606,4.311936,4.260654,4.014670,4.810606,4.570763,4.729299,4.062877,...,4.157066,4.594244,4.426700,4.375225,4.738746,4.001820,4.249442,4.687196,4.381240,4.431346
3,1836183.0,10.0,7.186322,7.098102,7.404546,7.123953,7.023080,7.232701,7.106642,7.115494,...,7.281785,7.420169,7.239500,7.175635,7.176322,7.244782,7.095404,7.051466,7.184079,7.166826
4,1653464.0,3.0,6.614349,6.763354,6.188758,6.426235,6.436756,6.757833,6.636064,6.288362,...,6.647743,6.633674,6.476090,6.454052,6.937150,6.889850,6.634454,6.484830,6.549337,6.739154


In [153]:
middle_preidiction_df['Predict_y'] = middle_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [154]:
middle_preidiction_df['Predict_y'].head()

0    6.473778
1    3.363153
2    4.386259
3    7.140422
4    6.540275
Name: Predict_y, dtype: float64

In [155]:
middle_preidiction_df = middle_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [158]:
middle_preidiction_df['index'] = middle_preidiction_df['index'].astype(int)

In [159]:
middle_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1469152,7.0,6.473778
1,831522,5.0,3.363153
2,1938251,2.0,4.386259
3,1836183,10.0,7.140422
4,1653464,3.0,6.540275
5,2335499,7.0,7.483874
6,1538044,8.0,8.434202
7,1461202,9.0,8.052614
8,727466,5.0,4.711369
9,938061,3.0,3.788787


In [160]:
middle_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbYoungModelMiddlePrediction_v1.parquet'))

### Young Model Old Prediction

In [161]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
old_preidiction_df = pd.DataFrame(np.array(o_prediction_list).T, columns = column_list)

In [162]:
old_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1535061.0,2.0,4.859666,4.517843,4.819750,4.488906,5.468989,4.687627,4.684260,5.132227,...,5.176656,4.723966,5.140882,5.143496,4.922386,4.693714,5.018629,4.654611,4.744168,4.950533
1,718598.0,5.0,5.423470,5.561956,5.081923,5.581120,5.345652,5.647656,5.232212,5.551328,...,5.286596,5.241395,5.268126,5.605401,5.535441,5.653308,5.379376,5.341910,4.818251,5.844172
2,1701982.0,7.0,6.977139,7.155666,6.765546,7.043703,6.922057,6.879908,7.018576,6.712383,...,6.961285,6.878421,7.054697,6.807316,6.975250,6.902295,7.017956,7.135243,6.906432,6.980763
3,1578430.0,7.0,6.518270,6.347220,6.539983,6.768547,6.594794,6.531702,6.880837,6.607730,...,6.134996,6.586335,6.695472,6.772330,6.360747,6.829571,6.186422,6.713863,6.806267,6.883454
4,1398218.0,10.0,7.232658,7.159914,7.133436,7.192966,7.371321,7.333807,7.928037,7.567041,...,7.235617,7.292641,7.488269,7.364222,7.498442,7.390056,7.568120,7.484967,7.463628,7.379086


In [163]:
old_preidiction_df['Predict_y'] = old_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [164]:
old_preidiction_df['Predict_y'].head()

0    4.872291
1    5.456793
2    6.908523
3    6.567139
4    7.374825
Name: Predict_y, dtype: float64

In [165]:
old_preidiction_df = old_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [166]:
old_preidiction_df['index'] = old_preidiction_df['index'].astype(int)

In [167]:
old_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1535061,2.0,4.872291
1,718598,5.0,5.456793
2,1701982,7.0,6.908523
3,1578430,7.0,6.567139
4,1398218,10.0,7.374825
5,1581052,5.0,5.150577
6,1764291,7.0,4.822250
7,1322632,7.0,4.680051
8,2573377,8.0,5.129573
9,865858,8.0,6.854324


In [168]:
old_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbYoungModelOldPrediction_v1.parquet'))

### Young Model Young Prediction

In [183]:
merged_young_prediction_df = None
for young_prediction_df in fold_predictions_array_list:
    young_prediction_df['index'] = young_prediction_df['index'].astype(int)
    young_prediction_df['Real_y'] = young_prediction_df['Real_y'].astype(int)
    print(young_prediction_df.columns)
    if merged_young_prediction_df is None:
        merged_young_prediction_df = young_prediction_df
    else:
        merged_young_prediction_df = pd.merge(merged_young_prediction_df, young_prediction_df, on=['index', 'Real_y'])

Index(['index', 'Real_y', 'Predict_y_0'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_1'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_2'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_3'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_4'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_5'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_6'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_7'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_8'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_9'], dtype='object')


In [185]:
merged_young_prediction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,Predict_y_8,Predict_y_9
0,1976009,7,5.120534,5.406739,5.057237,5.432996,5.472575,5.458484,5.691365,5.438991,5.572161,5.248793
1,1036932,3,3.924006,4.033377,3.818323,3.741839,3.898215,3.811231,3.727642,4.018414,3.618124,4.086395
2,2505821,4,3.754618,4.032731,3.819306,4.078354,3.860826,3.948386,3.880457,3.867070,4.122746,3.963184
3,1135079,1,4.022937,4.027075,3.741352,3.909735,4.231300,4.010872,3.856867,4.215916,4.073913,4.033562
4,2386018,6,7.990756,7.767663,7.807002,7.808393,7.637747,7.643322,7.918462,7.862862,7.915307,8.007626


In [186]:
merged_young_prediction_df['Predict_y'] = merged_young_prediction_df.iloc[:,2:13].mean(axis = 1)

In [187]:
merged_young_prediction_df = merged_young_prediction_df[['index', 'Real_y', 'Predict_y']]

In [188]:
merged_young_prediction_df.head(10)

,index,Real_y,Predict_y
0,1976009,7,5.389988
1,1036932,3,3.867757
2,2505821,4,3.932768
3,1135079,1,4.012353
4,2386018,6,7.835914
5,1493327,5,4.766764
6,549924,1,5.757768
7,2186766,7,6.240618
8,1936498,4,4.516366
9,2235916,7,4.263747


In [189]:
merged_young_prediction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbYoungModelYoundPrediction_v1.parquet'))

### Middle Hyperparameter Reload, build model, and conduct 10 fold 10 times CV

In [190]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [191]:
bayes_search_young = load('Results/BayesSearchAgeMiddle20iter.joblib')

In [192]:
print(f"Best parameters: {bayes_search_young.best_params_}")
print(f"Best score: {bayes_search_young.best_score_}")

Best parameters: OrderedDict([('gamma', 0.05965055817863728), ('learning_rate', 0.016150250040658724), ('max_delta_step', 0.10042448556024319), ('max_depth', 13), ('min_child_weight', 1.7624628686422215), ('n_estimators', 3304), ('reg_alpha', 10.0), ('reg_lambda', 0.002893997263318717), ('subsample', 0.8956214250879957)])
Best score: 0.40948431281386427


In [193]:
dict(bayes_search_young.best_params_)

{'gamma': 0.05965055817863728,
 'learning_rate': 0.016150250040658724,
 'max_delta_step': 0.10042448556024319,
 'max_depth': 13,
 'min_child_weight': 1.7624628686422215,
 'n_estimators': 3304,
 'reg_alpha': 10.0,
 'reg_lambda': 0.002893997263318717,
 'subsample': 0.8956214250879957}

In [194]:
middle_xgb = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True, kwargs = dict(bayes_search_young.best_params_))

In [195]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [196]:
scores = cross_val_score(young_xgb, Xm, ym, cv=kf, scoring='r2')

In [197]:
scores

array([0.39801738, 0.40378991, 0.3970049 , 0.39802149, 0.39907406,
       0.39465026, 0.3947646 , 0.39397942, 0.39150265, 0.40432979])

In [200]:
Xm.head()

,wave,INCOME_2,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,Smile,...,Corruption_business,Corruption_government,Performance_leadership,Gender_female,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1469152,10,35602.470662,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,0.0,0.0,1.0,5.0,0.0,2.0,3.0,NZL
831522,6,4357.352396,1.0,0.0,-1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,2.0,6.0,0.0,4.0,4.0,SDN
1938251,13,7202.172618,2.0,1.0,-1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,3.0,1.0,0.0,3.0,2.0,BIH
1836183,12,52643.260274,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,6.0,0.0,2.0,5.0,MLT
1653464,11,2561.527818,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,5.0,6.0,1.0,3.0,1.0,CHL


In [201]:
ym.head()

1469152     7.0
831522      5.0
1938251     2.0
1836183    10.0
1653464     3.0
Name: Cantril_ladder, dtype: float64

In [202]:
random.seed(42)

In [203]:
range_limit = 8192
num_samples = 10
sampled_integers = random.sample(range(range_limit), num_samples)
print(sampled_integers)

[1824, 409, 4506, 4012, 3657, 2286, 1679, 1424, 6912, 520]


In [204]:
X = Xm
y = ym
model = middle_xgb

In [205]:
y_prediction_list = [np.array(yy.index), np.array(yy.to_list())]
o_prediction_list = [np.array(yo.index), np.array(yo.to_list())]
fold_predictions_array_list = []
r2_scores_list = []
r2y_scores_list = []
r2o_scores_list = []

for i, random_integer in enumerate(sampled_integers):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_integer)
    
    r2_scores = []
    r2y_scores = []
    r2o_scores = []
    fold_predictions = []
    # Perform cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        this_folder = np.array([y_test.index, y_test, y_pred]).T
        fold_predictions.append(this_folder)
    
        # Young Predict
        yy_pred = model.predict(Xy)
        y_prediction_list.append(yy_pred)
        r2y = r2_score(yy, yy_pred)
        r2y_scores.append(r2y)
        
        # Old Predict
        yo_pred = model.predict(Xo)
        o_prediction_list.append(yo_pred)
        r2o = r2_score(yo, yo_pred)
        r2o_scores.append(r2o)
    
    print(f"Mean R2: {np.mean(r2_scores)}")
    print(f"Standard Deviation of R2: {np.std(r2_scores)}")
    print(f"R2 list: {r2_scores}")
    print(f"R2 Young list: {r2y_scores}")
    print(f"R2 Old list: {r2o_scores}")
    
    fold_predictions_array = pd.DataFrame(np.concatenate(fold_predictions, axis = 0), columns = ['index', 'Real_y', f'Predict_y_{i}'])
    fold_predictions_array_list.append(fold_predictions_array)
    r2_scores_list.append(r2_scores)
    r2y_scores_list.append(r2y_scores)
    r2o_scores_list.append(r2o_scores)

Mean R2: 0.3973804274900893
Standard Deviation of R2: 0.003446493089225287
R2 list: [0.3986668884919723, 0.3956818624115589, 0.4014568718610012, 0.3936731085949131, 0.3950415183814817, 0.3989115442355775, 0.4016805710631486, 0.39026959486116053, 0.39881854169873365, 0.39960377330134544]
R2 Young list: [0.3183108823569387, 0.31816791655313936, 0.3195398231572, 0.31850889039614183, 0.31849493336808754, 0.3172223237255716, 0.31807324833310535, 0.31864780042041274, 0.31901939274735147, 0.3179562936961381]
R2 Old list: [0.4017984454712469, 0.4006965345227592, 0.4011272475426846, 0.40176433813208035, 0.4018891610244065, 0.40116834078338504, 0.400647808297386, 0.40194400789134954, 0.4015528998488167, 0.40219433692600726]
Mean R2: 0.397110115151777
Standard Deviation of R2: 0.0018983470683116156
R2 list: [0.39726648530750175, 0.39501743129402744, 0.3981327441908262, 0.39896396550080326, 0.3970153501333823, 0.39758361941229414, 0.39318184394208977, 0.3955036501134386, 0.3993156502950751, 0.3991

### Middle Model Young Prediction

Prediction is average of 10 fold * 10 times

In [209]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
young_preidiction_df = pd.DataFrame(np.array(y_prediction_list).T, columns = column_list)

In [210]:
young_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1562723.0,5.0,4.621568,4.666876,4.654834,4.944523,5.042847,5.044880,5.140034,4.543129,...,4.979336,4.459723,4.650525,4.806470,4.934420,5.031139,4.686620,4.587959,4.786346,4.636636
1,708695.0,4.0,4.684507,4.317512,4.802039,4.307027,4.717960,4.785014,4.911132,4.334940,...,4.588868,4.507738,4.393626,4.403909,4.617470,4.831452,4.845782,4.529498,4.433863,4.358279
2,1828455.0,2.0,6.790520,6.269309,6.600958,6.319830,6.488219,6.486320,6.839993,6.469703,...,6.556964,6.474920,6.455927,6.641980,6.188348,6.353436,6.642846,6.495104,6.639003,6.517955
3,968859.0,8.0,5.211695,5.162162,5.919499,5.537360,4.945357,5.537626,5.515615,5.769351,...,5.499034,5.402608,5.482939,5.558377,5.489759,5.682730,5.515289,4.919305,5.316520,5.054358
4,2375401.0,5.0,4.429952,4.842068,5.236249,5.137251,4.918802,4.741779,5.004792,5.009373,...,4.787188,5.433940,5.162139,4.599927,4.941373,5.091188,4.604621,5.206207,4.619202,4.764240


In [211]:
young_preidiction_df['Predict_y'] = young_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [212]:
young_preidiction_df['Predict_y'].head()

0    4.820910
1    4.497302
2    6.505858
3    5.436034
4    4.974737
Name: Predict_y, dtype: float64

In [213]:
young_preidiction_df = young_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [214]:
young_preidiction_df['index'] = young_preidiction_df['index'].astype(int)

In [215]:
young_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1562723,5.0,4.820910
1,708695,4.0,4.497302
2,1828455,2.0,6.505858
3,968859,8.0,5.436034
4,2375401,5.0,4.974737
5,723699,6.0,6.453801
6,2069375,2.0,2.511315
7,1655669,10.0,4.426193
8,776224,7.0,5.641705
9,1002429,5.0,6.534380


In [216]:
young_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbMiddleModelYoungPrediction_v1.parquet'))

### Middle Model Old Prediction

In [217]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
old_preidiction_df = pd.DataFrame(np.array(o_prediction_list).T, columns = column_list)

In [218]:
old_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1535061.0,2.0,4.101773,4.393217,4.281288,3.806413,4.021470,4.056653,4.431487,3.980076,...,3.924871,4.124848,4.063640,3.860636,3.436504,4.038963,3.798092,4.091568,3.828469,4.040533
1,718598.0,5.0,5.306808,5.561452,5.598098,5.239180,5.362206,5.753240,5.392163,5.750226,...,5.694080,5.399746,5.474140,5.590003,5.379264,5.697059,5.422689,5.613432,5.320465,5.807169
2,1701982.0,7.0,6.660249,6.775815,6.906304,6.830786,6.679620,6.791051,6.651768,6.385378,...,6.666067,6.829247,6.758886,6.548736,6.727675,7.009635,6.853520,6.632224,6.838136,6.661746
3,1578430.0,7.0,6.834050,6.307304,6.642965,6.286050,6.323919,6.545181,6.670734,6.334890,...,6.608519,6.398882,6.474709,6.518171,6.597402,6.454708,6.430190,6.522000,6.898360,6.410652
4,1398218.0,10.0,6.961501,7.465508,7.172345,6.741922,7.275255,7.161025,7.452821,7.203275,...,7.105373,7.015994,7.241766,7.048047,7.011822,7.366494,6.976647,7.453019,7.584230,7.097053


In [219]:
old_preidiction_df['Predict_y'] = old_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [220]:
old_preidiction_df['Predict_y'].head()

0    4.036697
1    5.504358
2    6.711439
3    6.544661
4    7.196206
Name: Predict_y, dtype: float64

In [221]:
old_preidiction_df = old_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [222]:
old_preidiction_df['index'] = old_preidiction_df['index'].astype(int)

In [223]:
old_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1535061,2.0,4.036697
1,718598,5.0,5.504358
2,1701982,7.0,6.711439
3,1578430,7.0,6.544661
4,1398218,10.0,7.196206
5,1581052,5.0,4.365416
6,1764291,7.0,5.190850
7,1322632,7.0,4.787238
8,2573377,8.0,4.644580
9,865858,8.0,6.632134


In [224]:
old_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbMiddleModelOldPrediction_v1.parquet'))

### Middle Model Middle Prediction

In [225]:
merged_middle_prediction_df = None
for middle_prediction_df in fold_predictions_array_list:
    middle_prediction_df['index'] = middle_prediction_df['index'].astype(int)
    middle_prediction_df['Real_y'] = middle_prediction_df['Real_y'].astype(int)
    print(middle_prediction_df.columns)
    if merged_middle_prediction_df is None:
        merged_middle_prediction_df = middle_prediction_df
    else:
        merged_middle_prediction_df = pd.merge(merged_middle_prediction_df, middle_prediction_df, on=['index', 'Real_y'])

Index(['index', 'Real_y', 'Predict_y_0'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_1'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_2'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_3'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_4'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_5'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_6'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_7'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_8'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_9'], dtype='object')


In [226]:
merged_middle_prediction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,Predict_y_8,Predict_y_9
0,1469152,7,6.802451,6.763454,6.347287,6.746960,6.720069,6.535888,6.742231,6.420232,6.826801,6.237617
1,1761492,5,6.536808,5.969103,5.882705,6.168712,6.054534,6.019336,6.002455,5.722249,5.619442,6.141799
2,1605988,6,4.462887,4.377061,4.269424,4.361666,4.606577,4.566044,4.621729,4.428546,4.256291,4.576457
3,1027429,7,5.515312,5.577516,6.013709,5.515003,5.741398,5.842065,5.878135,5.724305,5.730289,5.679581
4,2225558,5,5.195260,4.688220,4.989563,4.609037,4.952246,5.239468,4.816686,5.014695,5.178007,4.590374


In [227]:
merged_middle_prediction_df['Predict_y'] = merged_middle_prediction_df.iloc[:,2:13].mean(axis = 1)

In [228]:
merged_middle_prediction_df = merged_middle_prediction_df[['index', 'Real_y', 'Predict_y']]

In [229]:
merged_middle_prediction_df.head(10)

,index,Real_y,Predict_y
0,1469152,7,6.614299
1,1761492,5,6.011714
2,1605988,6,4.452668
3,1027429,7,5.721731
4,2225558,5,4.927355
5,1761622,3,4.304742
6,2438374,4,3.543799
7,2482997,5,6.759255
8,537644,8,7.548979
9,618292,5,4.786225


In [230]:
merged_middle_prediction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbMiddleModelMiddlePrediction_v1.parquet'))

### Old Hyperparameter Reload, build model, and conduct 10 fold 10 times CV

In [231]:
class RandomRunNFoldsKFold(KFold):
    def __init__(self, n_splits=10, random_state=None, run_splits=3, **kwargs):
        super().__init__(n_splits=n_splits, shuffle=True, random_state=random_state, **kwargs)
        self.random_state = random_state
        self.actual_splits = run_splits  # Number of actual splits to use

    def split(self, X, y=None, groups=None):
        folds = list(super().split(X, y, groups))
        if self.random_state is not None:
            random.seed(self.random_state)
        selected_folds = random.sample(folds, self.actual_splits)
        for train_index, test_index in selected_folds:
            yield train_index, test_index

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.actual_splits

In [232]:
bayes_search_young = load('Results/BayesSearchAgeOld20iter.joblib')

In [233]:
print(f"Best parameters: {bayes_search_young.best_params_}")
print(f"Best score: {bayes_search_young.best_score_}")

Best parameters: OrderedDict([('gamma', 1.2045479993021058), ('learning_rate', 0.030685320506680522), ('max_delta_step', 0.3805781770349062), ('max_depth', 13), ('min_child_weight', 0.002361695637610135), ('n_estimators', 540), ('reg_alpha', 6.59041086695112), ('reg_lambda', 0.01642699361057668), ('subsample', 0.5572023993404673)])
Best score: 0.40462796837403414


In [234]:
dict(bayes_search_young.best_params_)

{'gamma': 1.2045479993021058,
 'learning_rate': 0.030685320506680522,
 'max_delta_step': 0.3805781770349062,
 'max_depth': 13,
 'min_child_weight': 0.002361695637610135,
 'n_estimators': 540,
 'reg_alpha': 6.59041086695112,
 'reg_lambda': 0.01642699361057668,
 'subsample': 0.5572023993404673}

In [235]:
old_xgb = xgb.XGBRegressor(objective='reg:squarederror',  device = 'cuda', tree_method='hist', random_state=42, enable_categorical=True, kwargs = dict(bayes_search_young.best_params_))

In [236]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [237]:
scores = cross_val_score(young_xgb, Xo, yo, cv=kf, scoring='r2')

In [238]:
scores

array([0.39121715, 0.39237264, 0.39046441, 0.39564496, 0.39348905,
       0.39613026, 0.38327939, 0.39280264, 0.39927281, 0.38978573])

In [239]:
Xo.head()

,wave,INCOME_2,Health_disable,Relative_have,Living_standard_change,Enough_food,Enough_shelter,Well_rested,Respected,Smile,...,Corruption_business,Corruption_government,Performance_leadership,Gender_female,Marital_status,Employment,Children_under15,Feeling_income,Income_level,COUNTRY_ISO3
1535061,10,4467.534252,1.0,1.0,-0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,5.0,6.0,0.0,3.0,3.0,MNE
718598,6,9792.918107,2.0,1.0,-0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,5.0,6.0,0.0,2.0,4.0,BLR
1701982,11,13238.653845,2.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,2.0,6.0,0.0,2.0,4.0,URY
1578430,11,10734.066166,1.0,1.0,-0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,1.0,5.0,6.0,0.0,2.0,3.0,CZE
1398218,9,24336.227568,2.0,1.0,-0.0,0.0,0.0,1.0,0.0,1.0,...,1.0,1.0,0.0,0.0,2.0,6.0,0.0,2.0,2.0,FIN


In [240]:
yo.head()

1535061     2.0
718598      5.0
1701982     7.0
1578430     7.0
1398218    10.0
Name: Cantril_ladder, dtype: float64

In [241]:
random.seed(42)

In [242]:
range_limit = 8192
num_samples = 10
sampled_integers = random.sample(range(range_limit), num_samples)
print(sampled_integers)

[1824, 409, 4506, 4012, 3657, 2286, 1679, 1424, 6912, 520]


In [243]:
X = Xo
y = yo
model = old_xgb

In [244]:
y_prediction_list = [np.array(yy.index), np.array(yy.to_list())]
m_prediction_list = [np.array(ym.index), np.array(ym.to_list())]
fold_predictions_array_list = []
r2_scores_list = []
r2y_scores_list = []
r2m_scores_list = []

for i, random_integer in enumerate(sampled_integers):
    kf = KFold(n_splits=10, shuffle=True, random_state=random_integer)
    
    r2_scores = []
    r2y_scores = []
    r2m_scores = []
    fold_predictions = []
    # Perform cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        r2 = r2_score(y_test, y_pred)
        r2_scores.append(r2)
        this_folder = np.array([y_test.index, y_test, y_pred]).T
        fold_predictions.append(this_folder)
    
        # Young Predict
        yy_pred = model.predict(Xy)
        y_prediction_list.append(yy_pred)
        r2y = r2_score(yy, yy_pred)
        r2y_scores.append(r2y)
        
        # Middle Predict
        ym_pred = model.predict(Xm)
        m_prediction_list.append(ym_pred)
        r2m = r2_score(ym, ym_pred)
        r2m_scores.append(r2m)
    
    print(f"Mean R2: {np.mean(r2_scores)}")
    print(f"Standard Deviation of R2: {np.std(r2_scores)}")
    print(f"R2 list: {r2_scores}")
    print(f"R2 Young list: {r2y_scores}")
    print(f"R2 Middle list: {r2m_scores}")
    
    fold_predictions_array = pd.DataFrame(np.concatenate(fold_predictions, axis = 0), columns = ['index', 'Real_y', f'Predict_y_{i}'])
    fold_predictions_array_list.append(fold_predictions_array)
    r2_scores_list.append(r2_scores)
    r2y_scores_list.append(r2y_scores)
    r2m_scores_list.append(r2m_scores)

Mean R2: 0.39201842859534936
Standard Deviation of R2: 0.007710901681397632
R2 list: [0.39577638478781163, 0.379160467531379, 0.4032633269463276, 0.39533306186218686, 0.392106259719633, 0.3853083447645723, 0.39280299993264, 0.37991878845238125, 0.3962826547255489, 0.400231997231013]
R2 Young list: [0.24560371906561718, 0.2506762072601608, 0.25058005318421206, 0.24960781811090593, 0.24690862107533695, 0.24612337629003067, 0.24809416836477982, 0.24644911806136394, 0.24761701925648893, 0.24356175963992255]
R2 Middle list: [0.3489049909205467, 0.3507904697963018, 0.3497004145794895, 0.3505001904317726, 0.3487486465299886, 0.3503698896841313, 0.34860030595749825, 0.35010815574595544, 0.34920030574531635, 0.3483976335103529]
Mean R2: 0.3913521529958978
Standard Deviation of R2: 0.006033410894183591
R2 list: [0.38829906014113813, 0.40045247925805894, 0.38516894081683184, 0.3977631439422683, 0.3842869678923746, 0.3915201653599375, 0.3854146861016443, 0.40129555392249494, 0.3884864960315443, 0.

### Old Model Young Prediction

Prediction is average of 10 fold * 10 times

In [245]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
young_preidiction_df = pd.DataFrame(np.array(y_prediction_list).T, columns = column_list)

In [246]:
young_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1562723.0,5.0,5.160886,5.065413,5.340314,5.249044,5.646032,4.520326,5.711357,5.999321,...,5.638526,5.917067,5.396200,5.366540,5.298741,5.590724,5.419779,6.033405,5.080078,6.417921
1,708695.0,4.0,4.409476,4.860135,4.750735,4.569668,4.643155,4.771224,5.165349,4.174999,...,5.135505,4.984783,4.448522,4.424931,5.233130,4.351461,4.472323,5.776587,4.920770,3.964070
2,1828455.0,2.0,8.121667,7.781330,8.241073,9.120522,8.382912,8.408014,8.323299,8.530830,...,8.506557,7.829419,8.562802,7.845344,7.777887,8.169092,8.320599,8.004613,7.121326,9.115368
3,968859.0,8.0,6.010443,5.455162,3.861758,5.669000,6.061444,5.102791,5.205057,5.063192,...,4.844675,5.459690,6.119966,5.460027,6.224106,4.649222,5.133681,5.410053,4.825837,6.240325
4,2375401.0,5.0,4.217713,5.773711,5.870012,5.290566,5.012790,5.893073,5.549117,5.944774,...,5.705350,5.897804,5.260014,5.427952,5.269232,5.108421,5.323901,5.519731,5.090106,5.913439


In [247]:
young_preidiction_df['Predict_y'] = young_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [248]:
young_preidiction_df['Predict_y'].head()

0    5.563618
1    4.874029
2    8.240769
3    5.240475
4    5.308108
Name: Predict_y, dtype: float64

In [249]:
young_preidiction_df = young_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [250]:
young_preidiction_df['index'] = young_preidiction_df['index'].astype(int)

In [251]:
young_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1562723,5.0,5.563618
1,708695,4.0,4.874029
2,1828455,2.0,8.240769
3,968859,8.0,5.240475
4,2375401,5.0,5.308108
5,723699,6.0,6.380922
6,2069375,2.0,3.571097
7,1655669,10.0,5.554096
8,776224,7.0,5.841195
9,1002429,5.0,6.965085


In [252]:
young_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbOldModelYoungPrediction_v1.parquet'))

### Old Model Middle Prediction

In [253]:
column_list = ['index', 'Real_y']
for num in list(range(0, 10*10)):
    column_list.append(f'Predict_y_{num}')
middle_preidiction_df = pd.DataFrame(np.array(m_prediction_list).T, columns = column_list)

In [254]:
middle_preidiction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,...,Predict_y_90,Predict_y_91,Predict_y_92,Predict_y_93,Predict_y_94,Predict_y_95,Predict_y_96,Predict_y_97,Predict_y_98,Predict_y_99
0,1469152.0,7.0,7.398458,7.097096,6.899435,7.072649,6.947849,6.840024,6.671311,7.424919,...,7.018598,7.190279,6.549906,6.443399,7.065905,7.216407,7.518769,6.434417,6.852788,7.302796
1,831522.0,5.0,2.677578,3.259015,2.611367,2.242530,4.318666,3.448470,2.913093,3.277819,...,2.992260,2.532118,2.772744,2.560140,2.565260,2.707935,2.388813,3.212473,3.665566,2.918628
2,1938251.0,2.0,4.393489,3.658326,4.224677,4.205077,4.401760,3.517339,4.164424,3.744359,...,3.848377,3.955189,4.756511,3.597142,4.805817,4.482926,3.849513,3.789284,3.804729,3.440333
3,1836183.0,10.0,7.220403,7.487572,7.579161,7.595115,7.164331,7.612072,7.442352,7.516885,...,7.816755,6.974221,7.549891,7.455636,7.128526,7.528386,7.357720,7.302977,7.676007,7.455504
4,1653464.0,3.0,6.377123,6.639279,6.411628,6.087641,6.125836,6.541431,6.375749,6.843594,...,6.429760,6.434082,6.453428,6.490562,6.190724,6.590322,6.651425,5.968759,7.194885,6.511088


In [255]:
middle_preidiction_df['Predict_y'] = middle_preidiction_df.iloc[:,2:103].mean(axis = 1)

In [256]:
middle_preidiction_df['Predict_y'].head()

0    7.068846
1    3.032391
2    3.928077
3    7.360205
4    6.429398
Name: Predict_y, dtype: float64

In [257]:
middle_preidiction_df = middle_preidiction_df[['index', 'Real_y', 'Predict_y']]

In [258]:
middle_preidiction_df['index'] = middle_preidiction_df['index'].astype(int)

In [259]:
middle_preidiction_df.head(10)

,index,Real_y,Predict_y
0,1469152,7.0,7.068846
1,831522,5.0,3.032391
2,1938251,2.0,3.928077
3,1836183,10.0,7.360205
4,1653464,3.0,6.429398
5,2335499,7.0,7.846463
6,1538044,8.0,8.489513
7,1461202,9.0,8.518427
8,727466,5.0,4.859102
9,938061,3.0,3.091933


In [260]:
middle_preidiction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbOldModelMiddlePrediction_v1.parquet'))

### Old Model Old Prediction

In [261]:
merged_old_prediction_df = None
for old_prediction_df in fold_predictions_array_list:
    old_prediction_df['index'] = old_prediction_df['index'].astype(int)
    old_prediction_df['Real_y'] = old_prediction_df['Real_y'].astype(int)
    print(old_prediction_df.columns)
    if merged_old_prediction_df is None:
        merged_old_prediction_df = old_prediction_df
    else:
        merged_old_prediction_df = pd.merge(merged_old_prediction_df, old_prediction_df, on=['index', 'Real_y'])

Index(['index', 'Real_y', 'Predict_y_0'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_1'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_2'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_3'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_4'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_5'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_6'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_7'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_8'], dtype='object')
Index(['index', 'Real_y', 'Predict_y_9'], dtype='object')


In [262]:
merged_old_prediction_df.head()

,index,Real_y,Predict_y_0,Predict_y_1,Predict_y_2,Predict_y_3,Predict_y_4,Predict_y_5,Predict_y_6,Predict_y_7,Predict_y_8,Predict_y_9
0,1618415,8,3.645731,4.399438,3.210601,4.702881,5.547719,4.069192,4.512550,4.056759,4.562299,4.122528
1,1688015,8,7.259276,6.929943,6.745081,6.808949,7.119973,7.406271,6.759096,7.278674,6.869382,6.568239
2,1726042,7,6.120153,6.395237,6.196351,6.202800,6.083681,6.194314,6.393667,6.451693,6.326237,6.659797
3,2461182,3,5.735749,6.220167,6.314794,6.337798,6.856579,6.853713,6.489721,5.894210,6.515608,6.229979
4,2308488,8,8.040057,8.112562,8.356316,8.586568,8.661208,8.406819,8.128919,8.732082,8.204692,8.542313


In [263]:
merged_old_prediction_df['Predict_y'] = merged_old_prediction_df.iloc[:,2:13].mean(axis = 1)

In [264]:
merged_old_prediction_df = merged_old_prediction_df[['index', 'Real_y', 'Predict_y']]

In [265]:
merged_old_prediction_df.head(10)

,index,Real_y,Predict_y
0,1618415,8,4.282970
1,1688015,8,6.974488
2,1726042,7,6.302393
3,2461182,3,6.344832
4,2308488,8,8.377154
5,2520496,5,4.808896
6,2587628,0,6.519768
7,2580220,4,4.617309
8,818170,8,7.868526
9,2568275,8,5.601995


In [266]:
merged_old_prediction_df.to_parquet(os.path.join('Results', 'PredictionWB_XgbOldModelOldPrediction_v1.parquet'))